In [12]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import base64
import dash_leaflet as dl
from CRUD import AnimalShelter
from bson import ObjectId


username = "aacuser"
password = "cs340"
db = AnimalShelter(username, password)


image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()


df = pd.DataFrame(columns=['breed', 'location_lat', 'location_long', 'name'])


app = JupyterDash('Andrew Obrochta')

app.layout = html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height': '100px'}),
    html.Center(html.B(html.H1('Andrew Obrochta CS-340 Dashboard'))),
    html.Hr(),
    
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'All', 'value': 'all'}
        ],
        value='all', 
        clearable=False
    ),
    
    html.Hr(),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=[],  
        filter_action='native',
        sort_action='native',
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        editable=True,
        row_selectable='single',
        selected_rows=[],
    ),
    
    html.Br(),
    html.Hr(),
    
    html.Div(id='map-id', className='col s12 m6', style={'width': '1000px', 'height': '500px'}),
    html.Div(id='graph-id', className='col s12 m6'),
    
    html.H4('Developed by [Andrew Obrochta]', style={'textAlign': 'center'}),
])


def transform_data(data):
    for item in data:
        for key, value in item.items():
            if isinstance(value, ObjectId):
                item[key] = str(value)
    return data


@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_table(filter_type):
    
    if filter_type == 'water':
        query = {"$and": [{"animal_type": "Dog"}, {"breed": {"$in": ["Labrador Retriever", "Newfoundland", "Chesapeake Bay Retriever"]}}]}
    elif filter_type == 'mountain':
        query = {"$and": [{"animal_type": "Dog"}, {"breed": {"$in": ["German Shepherd", "Belgian Malinois", "Bernese Mountain Dog"]}}]}
    elif filter_type == 'disaster':
        query = {"$and": [{"animal_type": "Dog"}, {"breed": {"$in": ["Bloodhound", "Border Collie", "Australian Cattle Dog"]}}]}
    else:
        query = {}

    
    filtered_data = db.read(query)
    
    
    filtered_data = transform_data(filtered_data)
    filtered_df = pd.DataFrame.from_records(filtered_data)
    
    # Return data for DataTable
    return filtered_df.to_dict('records')


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return [dcc.Graph(figure=px.pie(title="No Data Available"))]
    
    dffPie = pd.DataFrame.from_dict(viewData)
    if 'breed' not in dffPie.columns or dffPie['breed'].isnull().all():
        return [dcc.Graph(figure=px.pie(title="No Data Available"))]

    fig = px.pie(dffPie, names='breed', title='Preferred Animals')
    fig.update_layout(height=500, width=700)
    return [dcc.Graph(figure=fig)]


@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows"),
     Input('datatable-id', "derived_virtual_data")]
)
def update_map(selected_rows, viewData):
    if not selected_rows or not viewData:
        markerArray = (30.75, -97.48)
        toolTip = "Austin Animal Center"
        popUpHeading = "Austin Animal Center"
        popUpParagraph = "Shelter Home Location"
    else:
        dff = pd.DataFrame(viewData)
        coordLat = float(dff.iloc[selected_rows[0]]['location_lat'])
        coordLong = float(dff.iloc[selected_rows[0]]['location_long'])
        markerArray = (coordLat, coordLong)
        toolTip = dff.iloc[selected_rows[0]]['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff.iloc[selected_rows[0]]['name']

    return [dl.Map(style={'width': '700px', 'height': '450px'}, center=markerArray,
                   zoom=10, children=[
                       dl.TileLayer(id="base-layer-id"),
                       dl.Marker(position=markerArray, children=[
                           dl.Tooltip(toolTip),
                           dl.Popup([html.H1(popUpHeading), html.P(popUpParagraph)])
                       ])
                   ])
           ]


app.run_server(mode='external', debug=True, port=8081)

















Dash app running on http://127.0.0.1:8081/


In [7]:
print(df.columns)


Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')
